<a href="https://colab.research.google.com/github/JasonObeid/InteractiveVisualizations/blob/master/SummaryGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import urllib.request as urllib2
from urllib.request import urlopen
import json
from google.colab import drive
import os
import numpy as np
import pandas as pd
import math
from random import randint
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import traceback
from scipy.stats import linregress

#access files from drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
csv_dir = root_dir + 'chartSummarization-master/data/kong/data/'
json_dir = root_dir + 'chartSummarization-master/data/kong/specs/'
summary_dir = root_dir + 'chartSummarization-master/data/kong/summaries/'
#csv_dir = root_dir + 'chartSummarization-master/data/d3/data/'
#json_dir = root_dir + 'chartSummarization-master/data/d3/specs/'
#summary_dir = root_dir + 'chartSummarization-master/data/d3/summaries/'
csvPaths = []
jsonPaths = []

for filename in os.listdir(csv_dir):
  csvPaths.append(csv_dir+filename)
for filename in os.listdir(json_dir):
  jsonPaths.append(json_dir+filename)

csvPaths.sort()
jsonPaths.sort()
print(csvPaths)

Mounted at /content/gdrive
['/content/gdrive/My Drive/chartSummarization-master/data/kong/data/11.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/13.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/14.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/15.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/16.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/17.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/18.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/19.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/20.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/27.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/28.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kong/data/29.csv', '/content/gdrive/My Drive/chartSummarization-master/data/kon

In [0]:
def createSummary(jsonPath, csvPath):
  try:#open data
    with open(jsonPath) as json_file:
      loadedjson = json.load(json_file)
    loadedcsv = pd.read_csv(csvPath)
    xLabel = str(loadedjson['encoding']['x']['field'])
    yLabel = str(loadedjson['encoding']['y']['field'])
    dataY=loadedcsv[yLabel]
    dataX=loadedcsv[xLabel]
    size=str(len(dataX))
    fieldY=str(yLabel)
    fieldX=str(xLabel)
    chartType = loadedjson['mark']
    typeX=str(loadedjson['encoding']['x']['type'])
    typeY=str(loadedjson['encoding']['y']['type'])
    #find min and max
    maxValue=str(max(dataY))
    minValue=str(min(dataY))
    maxValueIndex=dataY.idxmax()
    minValueIndex=dataY.idxmin()
    #run bar
    summaryArray = []
    if(chartType == "bar"):
      summary1 = "This bar chart has " + size + " categories for the " + typeX + " data of " + xLabel + "."
      summary2 = " The highest value is found at " + str(dataX[maxValueIndex]) + " with a " + yLabel + " of " + str(dataY[maxValueIndex]) + "."
      summary3 = " The lowest value is found at " + str(dataX[minValueIndex]) + " with a " + yLabel + " of " + str(dataY[minValueIndex]) + "."
      summaryArray.append(summary1)
      summaryArray.append(summary2)
      summaryArray.append(summary3)
      trendsArray={"maxIndex":maxValueIndex,"minIndex":minValueIndex}
      dict_object = dict(summary=summaryArray, trends=trendsArray)
      json_name=jsonPath[65:len(jsonPath)-5]
      file_path=summary_dir + json_name + "_summary.json"
      file_object = open(file_path, 'w')
      # Save dict data into the JSON file.
      json.dump(dict_object, file_object)
    #run line
    if(chartType == "line"):
      #determine local trends
      trendArray = []
      i = 1
      #calculate variance between each adjacent y values
      while i < (len(dataY)):
        variance1 = dataY[i] - dataY[i-1]
        if(variance1 > 0):
          type1 = "positive"
        elif(variance1 < 0):
          type1 = "negative"
        else:
          type1 = "neutral"
        trendArray.append(type1)
        i = i + 1
      localTrends = []
      n = 0
      startIndex = 0
      trendLen = len(trendArray)
      #iterate through the variances and check for trends
      #creates dictionary containing the trend length, direction, start and end indices, and the linear regression of the trend
      while(n < trendLen):
        currentVal = trendArray[n-1]
        nextVal = trendArray[n]
        if(currentVal != nextVal or (currentVal == nextVal and n == (trendLen-1))):
          if(n == (trendLen-1)):
            endIndex = n+1
          else:
            endIndex = n
          xRange = dataX.loc[startIndex:endIndex]
          yRange = dataY.loc[startIndex:endIndex]
          result = linregress(xRange, yRange)
          slope = round(result[0],2)
          #normalize slope to between 0 and 1
          minimum = min(dataY)
          maximum = max(dataY)
          #called feature scaling /minmaxScaling
          scaler = preprocessing.MinMaxScaler()
          scaledX = scaler.fit_transform(xRange.values.reshape(-1, 1))
          scaledY = scaler.fit_transform(yRange.values.reshape(-1, 1))
          result2 = linregress(scaledX.reshape(1, -1), scaledY.reshape(1, -1))
          normalizedSlope = round(result2[0],2)
          if(abs(normalizedSlope) > 0.75):
            magnitude = "extremely"
          elif(abs(normalizedSlope) > 0.5 and abs(normalizedSlope) <= 0.75):
            magnitude ="strongly"
          elif(abs(normalizedSlope) > 0.25 and abs(normalizedSlope) <= 0.5):
            magnitude ="moderately"
          elif(abs(normalizedSlope) > 0 and abs(normalizedSlope) <= 0.25):
            magnitude = "slightly"
          intercept = round(result[1],2)
          trend = "y="+str(slope)+"x+"+str(intercept)
          trendRange = {"Length":(endIndex-startIndex+1),"direction":currentVal,"start":startIndex,"end":endIndex,"trend":trend,"magnitude":magnitude}
          localTrends.append(trendRange)
          startIndex = n
        n = n+1
      #sort the trend dictionaries by length
      sortedTrends = sorted(localTrends, key = lambda i: i['Length'], reverse=True)
      localTrendSummary = ""
      #determine the trend length which we consider to be significant (ex: if trend is longer than 1/4 of total length)
      significanceRange = round(len(dataY)/4)
      significantTrendCount = 0
      significantTrendArray = []
      #iterate through array of trend dictionaries, creating a new array with the trend dictionaries which are larger than the significance range
      for trends in sortedTrends:
        if (trends['Length'] > significanceRange):
          significantTrendCount = significantTrendCount + 1
          significantTrendArray.append(trends)
        #generate the textual summary from the significant trend dictionary array at m
      if (significantTrendCount > 0):
        m = 1
        startVal = str(dataX[(significantTrendArray[0]['start'])])
        endVal = str(dataX[(significantTrendArray[0]['end'])])
        direction = str(significantTrendArray[0]['direction'])
        magnitude = str(significantTrendArray[0]['magnitude'])
        #execute here if more than 1 significant trend
        similarSynonyms = ["Similarly", "Correspondingly", "Likewise", "Additionally", "Also", "In a similar manner"]
        contrarySynonyms = ["Contrarily", "Differently","On the other hand", "Conversely", "On the contrary", "Inversely", "But"]
        if (significantTrendCount > 1):
          extraTrends = ""
          localTrendSentence1 = "This chart has " + str(significantTrendCount) + " significant trends."
          summaryArray.append(localTrendSentence1)
          localTrendSummary = " The longest trend is " + magnitude + " " + direction + " which exists from " + startVal + " to " + endVal +"."
          summaryArray.append(localTrendSummary)
          while (m < significantTrendCount):
            if(direction == "positive"):
              length = len(similarSynonyms)
              random = randint(0, length-1)
              synonym = similarSynonyms[random]
              conjunction = synonym + ","
            elif(direction == "negative"):
              length = len(contrarySynonyms)
              random = randint(0, length-1)
              synonym = contrarySynonyms[random]
              conjunction = synonym + ","
            startVal = str(dataX[(significantTrendArray[m]['start'])])
            endVal = str(dataX[(significantTrendArray[m]['end'])])
            direction = str(significantTrendArray[m]['direction'])
            magnitude = str(significantTrendArray[m]['magnitude'])
            extraTrends = " " + conjunction + " the next significant trend is " + magnitude + " " + direction + " which exists from " + startVal + " to " + endVal +"."
            summaryArray.append(extraTrends)
            m = m + 1
        #execute here if only 1 significant trend
        else:
          localTrendSentence1 = "This chart has " + str(significantTrendCount) + " significant trend."
          summaryArray.append(localTrendSentence1)
          localTrendSummary = " This trend is " + magnitude + " "  + direction + " which exists from " + startVal + " to " + endVal +"."
          summaryArray.append(localTrendSummary)
      print(summaryArray)
      dict_object = dict(summary=summaryArray, trends=significantTrendArray)
      json_name=jsonPath[65:len(jsonPath)-5]
      file_path=summary_dir + json_name + "_summary.json"
      file_object = open(file_path, 'w')
      # Save dict data into the JSON file.
      json.dump(dict_object, file_object)
  except Exception as ex:
      print(ex)
    
#size = len(jsonPaths)
#i = 0
#while(i < size):
createSummary(jsonPaths[0],csvPaths[0])
  #i = i + 1

reduction operation 'argmax' not allowed for this dtype


In [0]:
outlierArray = []
    sorted(dataY)
    q1, q3 = np.percentile(dataY,[25,75])
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)
    for y in dataY:
      if y > upper_bound or y < lower_bound:
        outlierArray.append(y)